# WORKSHOP 6

### Conversione in PyPlasm del codice per la realizzazione di Santo Stefano Rotorno al Celio

In [26]:
from larlib import *
from math import sin
from math import cos


#### Misure di riferimento

In [27]:
r0=24.0
r2=48.0
r3=72.0
r4=88.0
wsteps=10.0
nsteps=10.0
wstep=wsteps/nsteps
hstep=1.0/3.0
hw4=29.0
hbasament=hstep*nsteps

#### Mappa cilindrica

In [28]:
def cylMap(bool=True):
    if bool:
        return MAP([lambda p: (p[1]) * COS(p[0]), lambda p: (p[1]) * SIN(p[0]), lambda p: (p[2])])
    else:
        return MAP([lambda p: (p[1]) * SIN(p[0]), lambda p: (p[1]) * COS(p[0]), lambda p: (p[2])])

#### Scala esterna

In [29]:
def vdom(h):
    return PROD([COMP([EMBED(1),INTERVALS(PI*3/24)])(3),QUOTE([h])])

def hdom(w):
    return COMP([S(2)(-1),EMBED(1)])(PROD([INTERVALS(PI*3/24)(3),QUOTE([w])]))

def steps(w,h):
    def steps1(n):
        return COMP([STRUCT,CAT,N(n)])([vdom(h),T(3)(h),hdom(w),T(2)(-w)])
    return steps1

hbasament=hstep*nsteps

stair = RIGHT([COMP([MKPOL,UKPOL])(steps(wstep, hstep)(nsteps)),COMP([S(2)(-1),SKEL_2,CUBOID])([PI/24,wsteps,hbasament])])
ramp = cylMap()(T(2)(r4)(stair))
stairs = STRUCT(NN(12)([ramp,R([1,2])(PI/6)]))

#### Basamento


In [30]:
basisSector=COMP([cylMap(False),EMBED(1)])(PROD([INTERVALS(2*PI/12)(4),INTERVALS(r4-(wstep*(nsteps-1)))(1)]))

basis=COMP([STRUCT,NN(12)])([basisSector,R([1,2])(2*PI/12)])

basement=COMP([R([1,2])(PI/-48),STRUCT])([stairs,T(3)(hstep*nsteps),basis])


In [31]:
VIEW(basement)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE41CF0> >

![basamento](basement.JPG)

#### Muro esterno


In [32]:
extWall2Da=INTERSECTION([MKPOL([[[0,0],[7,0],[7,5],[0,8],[7,3],[9,4],[10.5,1.5],[10.5,0],[11,1.5],[11,3]],
                                  [range(1,5),[2,8,7,6,5],[6,7,9,10]],[[1],[2],[3]]]),PROD([COMP([QUOTE,N(12)])(1),Q(8)])])

extWall2Db=INTERSECTION([MKPOL([[[0.5,0],[4,0],[5,0],[6,0],[6,3],[5.5,3.5],[5,3],[4.5,3.5],[4,3],[2,4],[0,3],
                                   [0,1.5],[0.5,1.5]],[[10,11,12,13],[1,2,9,10,13],[2,3,7,8,9],[3,4,5,6,7]],
                                  range(1,5)]),PROD([COMP([QUOTE,N(6)])(1),Q(4)])])

extWall2D=STRUCT([extWall2Da,T(1)(11),extWall2Db])

sizxExtWall=SIZE(1)(extWall2D)

extWall=R([2,3])(PI/2)(PROD([extWall2D,QUOTE([1.5])]))

curvedExtWall=COMP([cylMap(),T(2)(r3),S([1,3])([PI/(4*sizxExtWall),hw4/8])])(extWall)

doubleExtWall=STRUCT([curvedExtWall,S(1)(-1),curvedExtWall])

fullExtWall=COMP([STRUCT,NN(4)])([doubleExtWall,R([1,2])(PI/2)])


In [33]:
VIEW(STRUCT([basement,T(3)(hbasament), fullExtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE41E40> >

![muroesterno](muroesterno.JPG)

#### Colonne intermedie


In [34]:
def column(args):
    w,h=args
    basis=CUBOID([w,w,2.0*w/3.0])
    trunk=CYLINDER([w/2.0*0.85,h-w])(8)
    capitel=CUBOID([w,w,w/3.0])
    return TOP([TOP([basis,trunk]),capitel])

arcAngle=2.0*PI/50.4
wallAngle=-3.2*arcAngle/4.0
rotCross=3.2*arcAngle/4.0+2.5*arcAngle
hCol = 12.0

myColumn=COMP([MKPOL,UKPOL,T(2)(r2-0.75),column])([1.5,hCol])

the4cols=COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(4)])([R([1,2])(DIFF(arcAngle)),myColumn])

the5Cols=COMP([R([1,2])(arcAngle*0.4/3.2),MKPOL,UKPOL,STRUCT,NN(5)])([R([1,2])(DIFF(arcAngle)),myColumn])

theBotWal=COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,hCol])

theSecCols=STRUCT([R([1,2])(rotCross),theBotWal,R([1,2])(wallAngle),
                   the4cols,R([1,2])(-5*arcAngle),theBotWal,R([1,2])(wallAngle),the5Cols])

theMedColumns=COMP([STRUCT,NN(4)])([theSecCols,R([1,2])(PI/-2.0)])

In [35]:
VIEW(STRUCT([basement,T(3)(hbasament), fullExtWall,theMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE16E70> >

![colonneintermedie](colonneintermedie.JPG)

#### Archi sulle colonne

In [36]:
def bottomArc(d):
    return BEZIER(S1)([[0,0],[0,2*d/3],[d,2*d/3],[d,0]])

def topArc(d):
    return BEZIER(S1)([[0,2*d/3],[d,2*d/3]])

def arc2D(d):
    return BEZIER(S2)([bottomArc(d),topArc(d)])

def arc3D(d):
    def arc3D1(w):
        return COMP([T(2)(w),R([2,3])(PI/2)])(PROD([MAP(arc2D(3.2))(PROD([INTERVALS(1)(8),INTERVALS(1)(1)])), QUOTE([1.5])]))
    return arc3D1

def interarc(d1, d2):
    def interarc1(w):
        return CUBOID([d1,w,2*d2/3])
    return interarc1

def xarc(d1, d2):
    def xarc1(w):
        return RIGHT([RIGHT([interarc(d1,d2)(w),arc3D(d2)(w)]),interarc(d1,d2)(w)])
    return xarc1

theArc = xarc(0.4,3.2)(1.5)
sizeArc=SIZE(1)(theArc)
sx=(1/sizeArc)*arcAngle
sz=sx*(r2-2)

curvedArc=COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),S([1,3])([sx,sz])])(theArc)
heigthArc=SIZE(3)(curvedArc)

the5arcs=COMP([MKPOL,UKPOL,STRUCT,NN(5)])([curvedArc,R([1,2])(-arcAngle)])
the6arcs=COMP([MKPOL,UKPOL,STRUCT,NN(6)])([curvedArc,R([1,2])(-arcAngle)])
theMidWall=COMP([cylMap(),MKPOL,UKPOL,T(2)(r2-0.75),CUBOID])([3.2*arcAngle/4,1.5,heigthArc])
theSector = STRUCT([R([1,2])(rotCross),theMidWall, 
                    R([1,2])(wallAngle),the5arcs,R([1,2])(-5*arcAngle),theMidWall,R([1,2])(wallAngle),the6arcs])
theArcs=COMP([STRUCT,NN(4)])([theSector,R([1,2])(PI/-2)])
theArcsCols=STRUCT([theMedColumns,T(3)(hCol),theArcs])

In [37]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,theArcsCols]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE16F30> >

![archisucolonne](archisucolonne.JPG)

#### Muro intermedio superiore


In [38]:
medWall2D=INTERSECTION([MKPOL([[[0,4],[8,4],[17,4],[17,5],
                                [12.5,7.5],[8,5],[0,8]],[[1,2,6,7],[2,3,4,5,6]],
                                [[1],[2]]]),PROD([COMP([QUOTE,N(17)])(1),QUOTE([8])])])
sizxMedWall = SIZE(1)(medWall2D)

medWall=R([2,3])(PI/2)(PROD([medWall2D,QUOTE([1.5])]))

curvedMedWall=COMP([cylMap(False),MKPOL,UKPOL,T(2)(r2+0.75),S([1,3])([PI/(4*sizxMedWall),hw4/8])])(medWall)

doubleMedWall=STRUCT([curvedMedWall,S(1)(-1)(curvedMedWall)])

fullMedWall=COMP([STRUCT,NN(4)])([doubleMedWall,R([1,2])(PI/2)])

medWallArcs=STRUCT([fullMedWall,theArcsCols])

In [39]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE2D930> >

![murointermediosuperiore](murointermediosuperiore.JPG)

#### Muri laterali della croce

In [40]:
crossWall=COMP([T(1)(-1),R([1,2])(0.2*arcAngle),S(3)(hw4/8),CUBOID])([1.5,r3-r2,5])

singleWall=COMP([R([1, 2])(2.5 * arcAngle), T(2)(r2)])(crossWall)

doubleWall=STRUCT([singleWall,S(1)(-1),singleWall])

crossWalls=COMP([STRUCT,NN(4)])([doubleWall,R([1,2])(PI/2)])

#### Tetto sulla croce


In [41]:
halfTetto=MKPOL([[[0,r2,8],[8,r2,5],[8,r3,5],[0,r3,8]],[[1,2,3,4]],[1]])

curvedHalfTetto=COMP([cylMap(),S([1,3])([2.5*arcAngle/8,hw4/8])])(halfTetto)

doubleHalfTetto=STRUCT([curvedHalfTetto,S(1)(-1)(curvedHalfTetto)])

fullTetto=COMP([STRUCT,NN(4)])([doubleHalfTetto,R([1,2])(PI/-2)])

In [42]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,fullTetto]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE2DE40> >

![tettosullacroce](tettosullacroce.JPG)

#### Muro con Triplice apertura

In [43]:
def courtWall(d1, d2):
    def courtWall1(n1, n2):
        w=0.7
        tripleHole=TOP([STRUCT([column([w,2]),T(1)(2+w),column([w,2])]),xarc(2,2)(1)])
        h=SIZE(3)(tripleHole)
        op=ALIGN([[1,MAX,MIN],[2,MIN,MIN]])
        leftWall=PROD([COMP([QUOTE,N(n1)])(d1/n1),CUBOID([1,h])])
        rightWall=PROD([COMP([QUOTE,N(n2)])(d2/n2),CUBOID([1,h])])
        return op([op([leftWall,tripleHole]), rightWall])
    return courtWall1

In [44]:
VIEW(courtWall(6,6)(3,3))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEF47B0> >

![courtwall](courtwall.JPG)

#### Muro inferiore cortile interno

In [45]:
myCourtWall=courtWall(12,12)(9,9)

sizxCourtWall=SIZE(1)(myCourtWall)

mapping=COMP([cylMap(False),S([1,3])([(7.25*PI/24)*(1/sizxCourtWall),hw4/8])])

curvedCourtWall=COMP([OPTIMIZE,R([1,2])(-2.5*arcAngle),mapping,T(2)((r3+r2)/2)])(myCourtWall)

crossCourtWall=COMP([STRUCT,NN(4)])([curvedCourtWall,R([1,2])(PI/2)])

In [46]:
VIEW(STRUCT([fullTetto,crossWalls,crossCourtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEEB8D0> >

![cortileinterno](cortileinterno.JPG)

In [47]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,fullTetto,crossCourtWall,theMedColumns]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EE10EA0> >

![cortileinternocompleto](cortileinternocompleto.JPG)

#### Coronamento Muro Cortile interno

In [48]:
topCourtWall2D = INTERSECTION([MKPOL([[[0,3.333],[10,3.333],[10,4.333],[5,6.333],[0,4.333]],
                                      [range(1,5)],[[1]]]),PROD([COMP([QUOTE,N(10)])(1),QUOTE([8])])])
topCourtWall=R([2,3])(PI/2)(PROD([topCourtWall2D,QUOTE([1])]))

myTopCourtWall=RIGHT([topCourtWall,RIGHT([topCourtWall,topCourtWall])])

curvedTopCourtWall=COMP([R([1,2])(-2.5*arcAngle),mapping,T(2)((r3+r2+2)/2)])(myTopCourtWall)

fullTopCourtWall=COMP([STRUCT,NN(4)])([curvedTopCourtWall,R([1,2])(PI/2)])

In [49]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEF4660> >

![coronamento](coronamento.JPG)

#### Tetto su Cortile interno

In [50]:
rotWall=2.5*arcAngle

courTetto=INTERSECTION([MKPOL([[[0,(r3+r2+2)/2,3.333],[10,(r3+r2+2)/2,3.333],
                                [10,r2,4.333],[0,r2,4.333],[0,(r3+r2+2)/2,3.433],
                                [10,(r3+r2+2)/2,3.433],[10,r2,4.433],[0,r2,4.433]],[range(1,9)],[[1]]]),
                        PROD([COMP([QUOTE,N(10)])(1),PROD([QUOTE([(r3+r2+2)/2]),QUOTE([5])])])])

myCourTetto=RIGHT([RIGHT([courTetto,courTetto]), courTetto])

curvedCourTetto=COMP([R([1,2])(DIFF(rotWall)),mapping])(myCourTetto)

fullCourTetto=COMP([STRUCT,NN(4)])([curvedCourTetto,R([1,2])(PI/2)])

In [51]:
VIEW(STRUCT([fullExtWall,crossWalls,
             fullTetto,crossCourtWall,fullTopCourtWall,fullCourTetto]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEF4C30> >

![tetto](tetto.JPG)

In [52]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEF4810> >

![tettocompleto](tettocompleto.JPG)

#### Colonne muro interno

In [53]:
def radial22Obj(obj):
    the4Obj=COMP([STRUCT,NN(4)])([obj,R([1,2])(PI/11)])
    the2Obj=COMP([STRUCT,NN(2)])([obj,R([1,2])(PI/11)])
    the20Obj=COMP([STRUCT,NN(5)])([the4Obj,R([1,2])((4*PI)/11)])
    return STRUCT([the20Obj,R([1,2])((20*PI)/11),the2Obj])


internalCol=COMP([MKPOL,UKPOL,T(2)((r0-2)),column])([1.5,hCol])

the22Columns=radial22Obj(internalCol)

#### Travi muro interno

In [54]:
beam=PROD([QUOTE([-0.5,1.5,2,1.5]),PROD([QUOTE([2.2]),QUOTE([1.5])])])

curvedBeam=COMP([cylMap(),T([2,3])([(r0-2),hCol]),S(1)(PI/(11*5))])(beam)

the22Beams=radial22Obj(curvedBeam)

In [55]:
VIEW(the22Columns,the22Beams)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEFF6F0> >

![travi](travi.JPG)

In [56]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto,the22Columns,the22Beams]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EEFF3C0> >

![travicompleto](travicompleto.JPG)

#### Muro interno superiore

In [57]:
hwall=STRUCT([INSR(PROD)([QUOTE([-0.5,1.5,-2,1.5]),QUOTE([-0.3,1.4]), 
                          QUOTE([2.5*hCol])]),INSR(PROD)([QUOTE([-2.0,2]), 
                          QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])]),
                          INSR(PROD)([QUOTE([-2.25,1.5]),QUOTE([-0.3,1.4]),QUOTE([1.5*hCol,-0.5*hCol,0.5*hCol])])])

curvedHwall=COMP([cylMap(False),T([2,3])([r0-2,hCol]),S(1)(PI/(11*5))])(hwall)

the22Hwall=radial22Obj(curvedHwall)

In [58]:
VIEW(STRUCT([the22Columns,the22Beams,the22Hwall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0D030> >

![murointerno](murointerno.JPG)

In [59]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto,
             the22Columns,the22Beams,the22Hwall]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0D150> >

![murointernocompleto](murointernocompleto.JPG)

#### Coronamento superiore muro interno

In [60]:
myArc=MAP([COMP([SIN,S1]),COMP([COS,S1])])(QUOTE(N(5)(PI/10)))

vertex=MKPOL([[[0,0]],[[1]],[[1]]])

sector=JOIN([myArc,vertex])

triangle=S(1)(-1)(SIMPLEX(2))

sectorTriangle2D=STRUCT([T(1)(0.05),sector,T(1)(2),triangle])

doubleSect=STRUCT([sectorTriangle2D,S(1)(-1)(sectorTriangle2D)])

sect2D=S(1)(2 / 2.05)(doubleSect)

sect3D=R([2,3])(PI/2)(PROD([sect2D,QUOTE([0.8])]))

curvedSect=COMP([cylMap(False),T(2)(r0),S([1,3])([(2*PI)/(11*4),3])])(sect3D)

frieze=T(3)(3.5*hCol)(curvedSect)

def radial11Obj(obj):
    the2Obj=COMP([STRUCT,NN(2)])([obj,R([1,2])((2*PI)/11)])
    the10Obj=COMP([STRUCT,NN(5)])([the2Obj,R([1,2])((4*PI)/11)])
    return STRUCT([the10Obj,R([1,2])((20*PI)/11),obj])

the11Frieze=radial11Obj(frieze)

In [61]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto,
             the22Columns,the22Beams,the22Hwall,the11Frieze]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0D3F0> >

![coronamentocompleto](coronamentocompleto.JPG)

#### Traliccio Tetto interno

In [62]:
def trellis(v):
    H,L=v
    return COMP([R([2,3])(PI/2.0),EMBED(1),MKPOL])([[[0,0],[L/3,0],[2*L/3,0],[L,0],[2*L/3,H/3],[L/3,2*H/3],[0,H]],
         [[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,1],[1,6],[6,2],[2,5],[5,3]],
         [1,2,3,4,5,6,7,8,9,10,11]])

def radial(n):
    def radial1(obj):
        return COMP([STRUCT,NN(3)])([COMP([STRUCT,NN(n/3)])([obj,R([1,2])(2*PI/n)]),R([1,2])(2*PI/3)])
    return radial1

trellisTop3D=COMP([T(2)(-0.25),S(2)(0.5),OFFSET([1,1,1]),trellis])([hCol,r0-1])

theTopTrellis=T(3)(3.5*hCol)(trellisTop3D)

topTrellis = radial(12)(theTopTrellis)

In [63]:
VIEW(STRUCT([the11Frieze,topTrellis]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0D9C0> >

![traliccio](traliccio.JPG)

In [64]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto,
             the22Columns,the22Beams,the22Hwall,the11Frieze,topTrellis]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0D2D0> >

![tralicciocompleto](tralicciocompleto.JPG)

#### Traliccio Tetto intermedio

In [65]:
trellisMed3D=COMP([T(2)(-0.25),S(2)(0.5),OFFSET([1,1,1]),trellis])([hCol,r2-r0])

theMedTrellis=T(3)(hCol+3)(trellisMed3D)

medTrellis=radial(24)(T(1)(r0)(theMedTrellis))

doubleMedTrellis=STRUCT([medTrellis,R([1,2])(PI/24),medTrellis])

#### Santo Stefano Rotondo al Celio


In [66]:
VIEW(STRUCT([basement,T(3)(hbasament),fullExtWall,medWallArcs,crossWalls,
             fullTetto,crossCourtWall,theMedColumns,fullTopCourtWall,fullCourTetto,
             the22Columns,the22Beams,the22Hwall,the11Frieze,topTrellis,doubleMedTrellis]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000EF0DAE0> >

![santostefanorotondo](santostefanorotondo.JPG)